In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='sklearn_regression_model.pkl',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.24.1',             # Version of scikit-learn used to create the model.
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

In [ ]:
%%writefile "requirements_env.txt"

azureml-defaults
azureml-core
inference-schema[numpy-support]
numpy
scikit-learn==0.24.1
scipy
joblib


In [ ]:
from azureml.core import Environment

environment=Environment.from_pip_requirements('my-sklearn-environment', file_path="requirements_env.txt")
environment.register(ws)

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoring_script.py', 
                                   source_directory='.',
                                   environment=environment)

In [ ]:

from azureml.exceptions import ComputeTargetException
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration(cluster_purpose="DevTest")

aks_name = 'my-aks-dev' 
try:
    aks_target = None
    print('Found existing cluster, use it.')
except ComputeTargetException:
# Create the cluster
    aks_target = None
if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)



In [ ]:
# deploying the model and create a new endpoint
from azureml.core.webservice import AksEndpoint
import string
import random
from azureml.core.compute import ComputeTarget

#select a created compute
namespace_name="default"
# define the endpoint name
endpoint_name = f"aks{''.join(random.choice(string.ascii_lowercase) for _ in range(5))}"[:32]
# define the service name
print(endpoint_name)
version_name= "production"

endpoint_deployment_config = None

endpoint = None
endpoint.wait_for_deployment(True)

In [ ]:
version_name_add="variant" 

endpoint.create_version()
endpoint.wait_for_deployment(True)


In [ ]:
import json
test_sample = json.dumps({'data': [
    [1,2,3,4,5,6,7,8,9,10], 
    [10,9,8,7,6,5,4,3,2,1]
]})

test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

In [ ]:
endpoint.delete_version(version_name=version_name)
endpoint.wait_for_deployment(True)

In [ ]:
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

In [ ]:
endpoint.delete()